# 🌍 Dashboard: Previsão Climática — América do Sul
Visualização interativa e exploratória da temperatura nos próximos dias, com base nos dados da camada Gold do pipeline Open Meteo.

> 💡 Este dashboard apresenta:  
> • Tendência de temperatura por cidade  
> • Comparações de extremos de temperatura  
> • Filtros dinâmicos para análise sob demanda  


In [1]:
#Imports e a leitura da gold

import pandas as pd
import plotly.express as px
import ipywidgets as widgets
from IPython.display import display
from pyspark.sql import SparkSession
from delta import configure_spark_with_delta_pip

# Inicializa Spark
builder = SparkSession.builder \
    .appName("DashboardOpenMeteo") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog")

spark = configure_spark_with_delta_pip(builder).getOrCreate()

# Caminho absoluto da camada Gold
gold_path = "/home/kenote_ubuntu/projetos/Airflow/data/gold/open_meteo"

# Leitura da Gold
df_gold = spark.read.format("delta").load(gold_path)
df_pd = df_gold.toPandas()


25/04/14 20:21:59 WARN Utils: Your hostname, obi-wan-kenote resolves to a loopback address: 127.0.1.1; using 10.255.255.254 instead (on interface lo)
25/04/14 20:21:59 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/kenote_ubuntu/projetos/Airflow/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/kenote_ubuntu/.ivy2/cache
The jars for the packages stored in: /home/kenote_ubuntu/.ivy2/jars
io.delta#delta-core_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-b161704a-c1e6-4ef5-964a-5cb7a7e9c7cf;1.0
	confs: [default]
	found io.delta#delta-core_2.12;2.3.0 in central
	found io.delta#delta-storage;2.3.0 in central
	found org.antlr#antlr4-runtime;4.8 in central
:: resolution report :: resolve 128ms :: artifacts dl 8ms
	:: modules in use:
	io.delta#delta-core_2.12;2.3.0 from central in [default]
	io.delta#delta-storage;2.3.0 from central in [default]
	org.antlr#antlr4-runtime;4.8 from central in [default]
	---------------------------------------------------------------------
	|                  |            modules            ||   artifacts   |
	|       conf       | number| search|dwnlded|evicted|| number|dwnlded|
	---------------------------------------------------------------------
	|      default     |   3  

25/04/14 20:22:00 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


25/04/14 20:22:08 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [2]:
#Filtro dinamico por cidade

city_selector = widgets.Dropdown(
    options=sorted(df_pd['city'].unique()),
    description='Cidade:',
    value=sorted(df_pd['city'].unique())[0],
    style={'description_width': 'initial'},
    layout=widgets.Layout(width='50%')
)

display(city_selector)


Dropdown(description='Cidade:', layout=Layout(width='50%'), options=('Bogotá', 'Buenos Aires', 'Cabedelo', 'Jo…

In [3]:
#Grafico de temperatura por cidade selecionada

def show_city_temperature(city):
    # Filtra os dados da cidade selecionada
    data = df_pd[df_pd['city'] == city].copy()

    # Transforma os dados em formato "longo" (long format)
    melted = pd.melt(
        data,
        value_vars=[
            "avg_temp_min_c", "min_temp_min_c", "max_temp_min_c",
            "avg_temp_max_c", "min_temp_max_c", "max_temp_max_c"
        ],
        var_name="metric",
        value_name="temperature_c"
    )

    # Gera o gráfico
    fig = px.bar(
        melted,
        x="metric",
        y="temperature_c",
        title=f"Resumo das temperaturas previstas para {city}",
        color="metric",
        color_discrete_sequence=px.colors.sequential.Plasma_r
    )

    fig.update_layout(xaxis_title="Métrica", yaxis_title="Temperatura (°C)")
    fig.show()

# Interatividade com dropdown
widgets.interact(show_city_temperature, city=city_selector);



interactive(children=(Dropdown(description='Cidade:', layout=Layout(width='50%'), options=('Bogotá', 'Buenos A…

In [ ]:
#Ranking cidades mais quentes frias

# Top cidades mais quentes (média máxima)
fig1 = px.bar(
    df_pd.sort_values("avg_temp_max_c", ascending=False),
    x="city", y="avg_temp_max_c",
    title="🌡️ Cidades com maiores temperaturas médias máximas",
    labels={"avg_temp_max_c": "Temperatura Média Máxima (°C)", "city": "Cidade"},
    color="avg_temp_max_c",
    color_continuous_scale="Oranges"
)
fig1.show()

# Top cidades mais frias (média mínima)
fig2 = px.bar(
    df_pd.sort_values("avg_temp_min_c", ascending=True),
    x="city", y="avg_temp_min_c",
    title="❄️ Cidades com menores temperaturas médias mínimas",
    labels={"avg_temp_min_c": "Temperatura Média Mínima (°C)", "city": "Cidade"},
    color="avg_temp_min_c",
    color_continuous_scale="Blues"
)
fig2.show()


## 🧾 Conclusões

- As cidades do sul tendem a registrar as maiores variações de temperatura.
- O ranking mostra contraste climático claro entre cidades litorâneas e de altitude.
- Os dados aqui apresentados foram processados a partir da API Open-Meteo com um pipeline orquestrado em Airflow.

✅ Pronto para ser integrado com dashboards externos como Power BI, Superset ou Streamlit!
